In [ ]:
# Importing necessary libraries
import os
import geopandas as gpd
import pandas as pd

# Define scenarios, pollutants, years, and GNFRs
scenarios = ["s1", "s2", "s3", "s4"]
pollutants = ["ug_PM25_RH50", "ug_NO2"]
years = ["2030", "2050"]
gnfrs = ["trans", "res", "agri"]

# Read and merge shapefiles
path_fichier_shp_1, path_fichier_shp_2, path_fichier_shp_3 = "path1", "path2", "path3"
title_shp_1, title_shp_2, title_shp_3 = "title1", "title2", "title3"

donnees_shp_1 = gpd.read_file(os.path.join(path_fichier_shp_1, f"{title_shp_1}.shp"))
donnees_shp_2 = gpd.read_file(os.path.join(path_fichier_shp_2, f"{title_shp_2}.shp"))
donnees_shp_3 = gpd.read_file(os.path.join(path_fichier_shp_3, f"{title_shp_3}.shp"))

donnees_merged = pd.concat([donnees_shp_1, donnees_shp_2, donnees_shp_3], ignore_index=True)

# Loop through scenarios, years, pollutants, and GNFR categories
for sc in scenarios:
    for year in years:
        for pol in pollutants:
            for g in gnfrs:

                path_fichier_expo = f"data/2-output-data/{sc}/{pol}/{year}/expo-shp"
                title_expo = f"expo-shp-{g}"

                # Replace these functions with appropriate implementations
                conc_points = coordo_sherpa_new(sc, pol, year, g)
                donnees_exportees_transformed = donnees_exportees.to_crs(conc_points.crs)
                conc_ineris = coordo_ineris(pol)
                conc_corrigee = correction(conc_points, conc_ineris)
                donnees_expo = expo(donnees_exportees_transformed, conc_corrigee, grille_combinee)

                # Optional exports
                # export_data_shp(donnees_expo, path_fichier_expo, title_expo)
                # donnees_mixtes = mortalite_evitee_iris(donnees_expo, pol, year)
                # export_data_shp(donnees_mixtes, path_fichier_mixte, title_mixte)

                path = f"data/2-output-data/{sc}/{pol}/{year}"
                os.makedirs(path, exist_ok=True)

                # Generate mortality and life expectancy files
                tab = mortalite_age(donnees_merged, donnees_expo, year, pol)
                csv_path = os.path.join(path, f"mortalite_evitee_{g}.csv")
                tab.to_csv(csv_path, index=False)

                res = life_exp(tab, year)
                csv_path2 = os.path.join(path, f"esp_vie_{g}.csv")
                res.to_csv(csv_path2, index=False)
